# Mask Processing from DICOM

In [3]:
from pathlib import Path
from damply import dirs
import SimpleITK as sitk

from readii.loaders import loadDicomSITK
from readii.image_processing import flattenImage

import numpy as np
import pandas as pd

from imgtools.coretypes.imagetypes.scan import read_dicom_scan
from imgtools.dicom import load_dicom

/Users/katyscott/Documents/BHKLab_GitHub/ispy2_r2r/.pixi/envs/default/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
mit_index = pd.read_csv(dirs.PROCDATA / "TCIA_ISPY2/images/mit_ISPY2/mit_ISPY2_index.csv")

mask_dicom_file = Path( dirs.RAWDATA / "TCIA_ISPY2/images/ISPY2/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/51900.000000-ISPY2 VOLSER uni-lateral cropped Analysis Mask-78091/1-1.dcm")
image_dicom_dir = Path( dirs.RAWDATA / "TCIA_ISPY2/images/ISPY2/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/51800.000000-ISPY2 VOLSER uni-lateral cropped original DCE-30523")

In [82]:
from imgtools.dicom.dicom_metadata import extract_metadata

mask_metadata = extract_metadata(mask_dicom_file)
mask_slice_ids = mask_metadata['ReferencedSOPUIDs']

In [83]:
image_slice_ids = []
image_indices = []

for image_dicom_file in sorted(image_dicom_dir.glob("*.dcm")):
    image_metadata = load_dicom(image_dicom_file, stop_before_pixels=True)
    image_indices.append(str(image_metadata.get("InstanceNumber", "")))
    image_slice_ids.append(str(image_metadata.get("SOPInstanceUID", "")))

In [ ]:
image_slices_pd = pd.Series(data=image_slice_ids, index=image_indices)

image_slices_pd[image_slices_pd.isin(mask_slice_ids)]

In [ ]:
image_slices_pd.iloc[40:50]

# Saving out fixed DICOM SEG

In [91]:
import highdicom as hd
from pydicom.sr.codedict import codes

image_files = image_dicom_dir.glob("*.dcm")

image_datasets = [hd.imread(str(f)) for f in image_files]

mask = hd.seg.segread(mask_dicom_file)

mask_pixels = mask.pixel_array

roi_only = np.where(mask_pixels == 0, 1, 0).astype(np.uint8)


In [ ]:
cropped_image = []
                 
for image_dicom_file in image_datasets:
    if image_dicom_file.SOPInstanceUID in mask_slice_ids:
        cropped_image.append(image_dicom_file)

In [96]:
len(cropped_image)

80

In [ ]:


algorithm_identification = hd.AlgorithmIdentificationSequence(
    name='Background Threshold, PE threshold and connectivity filter',
    version='v1.0',
    family=codes.cid7162.AdaptiveFiltering
)


tumour_segment_description = hd.seg.SegmentDescription(
    segment_number=1,
    segment_label='Tumor',
    segmented_property_category=codes.cid7150.Tissue,
    segmented_property_type=codes.cid7151.Breast,
    algorithm_type=hd.seg.SegmentAlgorithmTypeValues.SEMIAUTOMATIC,
    algorithm_identification=algorithm_identification
)

tumor_dataset = hd.seg.Segmentation(
    source_images = cropped_image,
    pixel_array=roi_only,
    segmentation_type=hd.seg.SegmentationTypeValues.BINARY,
    segment_descriptions=[tumour_segment_description],
    series_instance_uid=hd.UID(),
    series_number=2,
    sop_instance_uid=hd.UID(),
    instance_number=1,
    manufacturer="GE MEDICAL SYSTEMS",
    manufacturer_model_name="Signa HDxt",
    software_versions='220',
    device_serial_number='Device XYZ',
    series_description='Binarized VOLSER Analysis Mask',
    omit_empty_frames=False
)

In [104]:
out_path = dirs.PROCDATA / "TCIA_ISPY2/images/tumour_only_dicom_masks/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/51900.000000-ISPY2 VOLSER uni-lateral cropped Analysis Mask-78091/1-1.dcm"
out_path.parent.mkdir(parents = True, exist_ok= True)

tumor_dataset.save_as(dirs.PROCDATA / "TCIA_ISPY2/images/tumour_only_dicom_masks/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/51900.000000-ISPY2 VOLSER uni-lateral cropped Analysis Mask-78091/1-1.dcm")

In [ ]:
cropped_image

In [ ]:
hd.seg.Segmentation(
    source_images=image_datasets,
    pixel_array=tumour_only
)